In [1]:
from src_clean.dataloader.dataloader import DataLoader
from src_clean.preprocessor.prepro_new import PreprocessorTwo

In [2]:
train_df, test_df = DataLoader.load_train_test_dfs()
#train_df = PreprocessorTwo.preprocess(train_df, exclude_cols_to_drop=['srch_id', 'prop_id', 'position'])
train_df = PreprocessorTwo.remove_outliers(train_df)
train_df = PreprocessorTwo.relevance(train_df)
train_df = PreprocessorTwo.prepro_comp_columns(train_df)

#test_df = PreprocessorTwo.preprocess(test_df, exclude_cols_to_drop=['prop_id'])
test_df = PreprocessorTwo.prepro_comp_columns(test_df)
train_df, test_df = PreprocessorTwo.calculate_statistics(train_df, test_df)
train_df, test_df = PreprocessorTwo.mean_pos_per_prop_id(train_df, test_df)
#train_df, test_df = PreprocessorTwo.click_and_booking_prob(train_df, test_df)

train_df = PreprocessorTwo.drop_cols(train_df, exclude_cols_to_drop=['srch_id', 'prop_id'])
test_df = PreprocessorTwo.drop_cols(test_df)

In [3]:
train_df, test_df = PreprocessorTwo.impute(train_df, test_df)

In [11]:
X_train, _, y_train, _ = DataLoader.split_df_into_train_and_val_batches(train_df, validation_size=0)

In [12]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
import numpy as np

reg = SGDRegressor()
reg.fit(np.vstack(X_train), np.hstack(y_train))

SGDRegressor()

In [13]:
from src_clean.score.ndcg import calculate_ndcg
from src_clean.ranker.ranker import Ranker

pred = reg.predict(test_df.to_numpy())
ranker = Ranker()
ranking_file_path = ranker.make_ranking_from_prediction(
    pred, model_name="SGD_Regressor"
)
calculate_ndcg(ranking_file_path)

0.1462618289569216

In [14]:
pred

array([5.06361376e+15, 8.63637584e+15, 4.16362705e+15, ...,
       3.30707908e+15, 9.13777653e+15, 7.74089412e+15])